<a href="https://colab.research.google.com/github/cpython-projects/python_da_06_11_25/blob/main/lesson_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Що таке API

## API (Application Programming Interface)

* Це «програма для спілкування з програмою».
* Через API можна отримувати дані від зовнішніх сервісів.
* Часто використовується формат **JSON**.

## Типи API

* **Публічні** (без авторизації): можна одразу робити запит.
* **З авторизацією**: потрібен ключ (`api_key` або `token`) — як пароль.

## Де шукати API

* На сайтах сервісів → розділ *Developers* або *API Docs*
* Приклади:

  * [CoinGecko API (без ключа)](https://www.coingecko.com/en/api/documentation)
  * [CoinMarketCap API (потрібен ключ)](https://coinmarketcap.com/api/)
  * [OpenWeatherMap](https://openweathermap.org/api)
  * [NASA API](https://api.nasa.gov/)

## JSON — формат обміну даними в API

**JSON (JavaScript Object Notation)** — це текстовий формат передачі даних, який найчастіше повертають API.

Типовий JSON виглядає так:

```json
{
  "user_id": 101,
  "name": "Anna",
  "city": "Batumi",
  "orders": [
    {
      "order_id": 5001,
      "amount": 120.5,
      "status": "completed"
    },
    {
      "order_id": 5002,
      "amount": 75.0,
      "status": "pending"
    }
  ],
  "is_active": true
}
```

### Основні типи даних у JSON:

* `string` → `"Batumi"`
* `number` → `120.5`
* `boolean` → `true / false`
* `array` → `[ {...}, {...} ]`
* `object` → `{ "key": value }`
* `null`

> **Важливо:** JSON — це **рядок тексту**, а не Python-обʼєкт.


## Серіалізація і десеріалізація (ключове поняття)

### Серіалізація

**Серіалізація** — це процес **перетворення Python-обʼєкта у JSON-рядок**, щоб:

* передати його через API
* зберегти у файл
* відправити по мережі

In [ ]:
data = {
    "user_id": 101,
    "name": "Anna",
    "city": "Batumi"
}

In [ ]:
import json
json_string = json.dumps(data)
json_string

'{"user_id": 101, "name": "Anna", "city": "Batumi"}'


### Десеріалізація

**Десеріалізація** — це зворотній процес:
**JSON → Python-обʼєкт**

In [ ]:
json_string = '{"user_id": 101, "name": "Anna", "city": "Batumi"}'

python_data = json.loads(json_string)
python_data

{'user_id': 101, 'name': 'Anna', 'city': 'Batumi'}

## Як це виглядає при роботі з API

In [ ]:
import requests

url = "https://jsonplaceholder.typicode.com/users/1"
response = requests.get(url)

In [ ]:
data = response.json()
data

{'id': 1,
 'name': 'Leanne Graham',
 'username': 'Bret',
 'email': 'Sincere@april.biz',
 'address': {'street': 'Kulas Light',
  'suite': 'Apt. 556',
  'city': 'Gwenborough',
  'zipcode': '92998-3874',
  'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
 'phone': '1-770-736-8031 x56442',
 'website': 'hildegard.org',
 'company': {'name': 'Romaguera-Crona',
  'catchPhrase': 'Multi-layered client-server neural-net',
  'bs': 'harness real-time e-markets'}}

In [ ]:
type(data)

dict

In [ ]:
response.text

'{\n  "id": 1,\n  "name": "Leanne Graham",\n  "username": "Bret",\n  "email": "Sincere@april.biz",\n  "address": {\n    "street": "Kulas Light",\n    "suite": "Apt. 556",\n    "city": "Gwenborough",\n    "zipcode": "92998-3874",\n    "geo": {\n      "lat": "-37.3159",\n      "lng": "81.1496"\n    }\n  },\n  "phone": "1-770-736-8031 x56442",\n  "website": "hildegard.org",\n  "company": {\n    "name": "Romaguera-Crona",\n    "catchPhrase": "Multi-layered client-server neural-net",\n    "bs": "harness real-time e-markets"\n  }\n}'

## Перетворення JSON → Pandas DataFrame

Приклад API, що повертає список обʼєктів:

In [ ]:
data = [
  {"date": "2025-01-01", "revenue": 1200},
  {"date": "2025-01-02", "revenue": 1500},
  {"date": "2025-01-03", "revenue": 1100}
]

import pandas as pd

df = pd.DataFrame(data)
print(df)

         date  revenue
0  2025-01-01     1200
1  2025-01-02     1500
2  2025-01-03     1100


## Вкладені JSON

In [ ]:
data = {
  "user_id": 101,
  "orders": [
    {"order_id": 1, "amount": 120},
    {"order_id": 2, "amount": 75}
  ]
}

df = pd.json_normalize(data, record_path="orders")
df

,order_id,amount
0,1,120
1,2,75


# Робота з публічним API (CoinGecko)

**Знайдемо потрібний endpoint**

* Документація CoinGecko: [https://www.coingecko.com/en/api/documentation](https://www.coingecko.com/en/api/documentation)
* Обираємо: `/coins/markets`

```http
GET https://api.coingecko.com/api/v3/coins/markets
?vs_currency=usd&order=market_cap_desc&per_page=10&page=1
```

**Надішлемо запит через Python**

In [ ]:
import requests

url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 10,
    "page": 1
}

response = requests.get(url, params=params)
print(response.status_code)
data = response.json()

200


In [ ]:
print(*data, sep='\n')

{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 94666, 'market_cap': 1891788878710, 'market_cap_rank': 1, 'fully_diluted_valuation': 1891788878710, 'total_volume': 45983408204, 'high_24h': 96933, 'low_24h': 94369, 'price_change_24h': -1848.5993750153284, 'price_change_percentage_24h': -1.91536, 'market_cap_change_24h': -38731392731.826416, 'market_cap_change_percentage_24h': -2.00627, 'circulating_supply': 19976631.0, 'total_supply': 19976631.0, 'max_supply': 21000000.0, 'ath': 126080, 'ath_change_percentage': -24.91581, 'ath_date': '2025-10-06T18:57:42.558Z', 'atl': 67.81, 'atl_change_percentage': 139506.92693, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2026-01-16T17:13:51.167Z'}
{'id': 'ethereum', 'symbol': 'eth', 'name': 'Ethereum', 'image': 'https://coin-images.coingecko.com/coins/images/279/large/ethereum.png?1696501628', 'current_price': 3271

In [ ]:
print(data[0])

{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 112996, 'market_cap': 2250439626317, 'market_cap_rank': 1, 'fully_diluted_valuation': 2250439626317, 'total_volume': 42029614551, 'high_24h': 113226, 'low_24h': 111200, 'price_change_24h': 1752.13, 'price_change_percentage_24h': 1.57504, 'market_cap_change_24h': 34299788862, 'market_cap_change_percentage_24h': 1.54773, 'circulating_supply': 19918300.0, 'total_supply': 19918300.0, 'max_supply': 21000000.0, 'ath': 124128, 'ath_change_percentage': -8.84696, 'ath_date': '2025-08-14T00:37:02.582Z', 'atl': 67.81, 'atl_change_percentage': 166760.57359, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2025-09-09T07:53:01.198Z'}


**Перетворимо у DataFrame**

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df[['id', 'symbol', 'current_price', 'market_cap', 'price_change_percentage_24h']]

,id,symbol,current_price,market_cap,price_change_percentage_24h
0,bitcoin,btc,94666.000000,1891788878710,-1.91536
1,ethereum,eth,3271.380000,394972111950,-1.55699
2,tether,usdt,0.999621,186799685036,-0.00517
3,binancecoin,bnb,928.280000,126705671339,-1.09255
4,ripple,xrp,2.040000,123957906224,-2.18476
5,solana,sol,141.580000,80101905246,-1.12225
6,usd-coin,usdc,1.000000,75686837868,0.03005
7,staked-ether,steth,3272.720000,29304591657,-1.48911
8,tron,trx,0.307618,29058975028,-0.53469
9,dogecoin,doge,0.136096,22947704164,-3.82133


# Робота з API з авторизацією (CoinMarketCap)

## Реєстрація та отримання ключа

* Зареєструйтесь на [CoinMarketCap](https://coinmarketcap.com/api/)
* Після реєстрації отримаєте `X-CMC_PRO_API_KEY`

## Приклад документації:

* Документація: [https://coinmarketcap.com/api/documentation/v1/](https://coinmarketcap.com/api/documentation/v1/)
* Endpoint: `/v1/cryptocurrency/listings/latest`

## Приклад запиту з ключем

In [ ]:
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
headers = {
    "Accepts": "application/json",
    "X-CMC_PRO_API_KEY": "17ad748a-f734-46e1-8ae0-16044d2215b8"
}
params = {
    "start": "1",
    "limit": "100",
    "convert": "USD"
}

response = requests.get(url, headers=headers, params=params)
data = response.json()

In [ ]:
response.status_code

200

In [ ]:
data.keys()

dict_keys(['status', 'data'])

## Як зрозуміти структуру JSON?

In [ ]:
import json
print(json.dumps(data, indent=2))

{
  "status": {
    "timestamp": "2025-09-09T07:54:09.850Z",
    "error_code": 0,
    "error_message": null,
    "elapsed": 39,
    "credit_count": 1,
    "notice": null,
    "total_count": 9514
  },
  "data": [
    {
      "id": 1,
      "name": "Bitcoin",
      "symbol": "BTC",
      "slug": "bitcoin",
      "num_market_pairs": 12336,
      "date_added": "2010-07-13T00:00:00.000Z",
      "tags": [
        "mineable",
        "pow",
        "sha-256",
        "store-of-value",
        "state-channel",
        "coinbase-ventures-portfolio",
        "three-arrows-capital-portfolio",
        "polychain-capital-portfolio",
        "binance-labs-portfolio",
        "blockchain-capital-portfolio",
        "boostvc-portfolio",
        "cms-holdings-portfolio",
        "dcg-portfolio",
        "dragonfly-capital-portfolio",
        "electric-capital-portfolio",
        "fabric-ventures-portfolio",
        "framework-ventures-portfolio",
        "galaxy-digital-portfolio",
        "huobi-capit

## Витягуємо корисні дані у DataFrame

In [ ]:
coins = data['data']  # список словників
print(type(coins), len(coins))

<class 'list'> 100


In [ ]:
coins[0]

{'id': 1,
 'name': 'Bitcoin',
 'symbol': 'BTC',
 'slug': 'bitcoin',
 'num_market_pairs': 12518,
 'date_added': '2010-07-13T00:00:00.000Z',
 'tags': ['mineable',
  'pow',
  'sha-256',
  'store-of-value',
  'state-channel',
  'coinbase-ventures-portfolio',
  'three-arrows-capital-portfolio',
  'polychain-capital-portfolio',
  'binance-labs-portfolio',
  'blockchain-capital-portfolio',
  'boostvc-portfolio',
  'cms-holdings-portfolio',
  'dcg-portfolio',
  'dragonfly-capital-portfolio',
  'electric-capital-portfolio',
  'fabric-ventures-portfolio',
  'framework-ventures-portfolio',
  'galaxy-digital-portfolio',
  'huobi-capital-portfolio',
  'alameda-research-portfolio',
  'a16z-portfolio',
  '1confirmation-portfolio',
  'winklevoss-capital-portfolio',
  'usv-portfolio',
  'placeholder-ventures-portfolio',
  'pantera-capital-portfolio',
  'multicoin-capital-portfolio',
  'paradigm-portfolio',
  'bitcoin-ecosystem',
  'layer-1',
  'ftx-bankruptcy-estate',
  '2017-2018-alt-season',
  'us-st

In [ ]:
df = pd.DataFrame([{
    "name": coin["name"],
    "symbol": coin["symbol"],
    "price": coin["quote"]["USD"]["price"],
    "market_cap": coin["quote"]["USD"]["market_cap"],
    "change_24h": coin["quote"]["USD"]["percent_change_24h"]
} for coin in coins])

df

,name,symbol,price,market_cap,change_24h
0,Bitcoin,BTC,94538.229542,1.888555e+12,-2.100819
1,Ethereum,ETH,3265.990863,3.941875e+11,-1.727879
2,Tether USDt,USDT,0.999428,1.867600e+11,-0.001565
3,BNB,BNB,928.083223,1.265546e+11,-1.143419
4,XRP,XRP,2.042228,1.239613e+11,-2.058921
...,...,...,...,...,...
95,ether.fi,ETHFI,0.730745,4.784644e+08,-1.768493
96,Celestia,TIA,0.543902,4.719132e+08,-3.736619
97,Decred,DCR,27.374142,4.715967e+08,6.964793
98,Humanity Protocol,H,0.203368,4.688364e+08,-0.729933


## Як розуміти документацію:

* `GET` — означає, що потрібно робити GET-запит
* У дужках `required` / `optional` — обов’язковий або необов’язковий параметр
* Тип даних: `string`, `int`, `float`

## Як відлагоджувати:

* `response.status_code != 200` — означає помилку (https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Status)
* `response.text` — покаже текст помилки
* `try-except` — ловити помилки

## Отримати топ-10 криптовалют за ринковою капіталізацією

## Відфільтрувати монети з падінням більше ніж на 1%


## Побудувати barplot